# 1. Installation

In [41]:
%pip install -q peft==0.4.0 datasets transformers torch accelerate --upgrade
%pip install wandb -q


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 2. Imports

In [42]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, PeftModel
import torch
import os
import wandb 
from transformers import TrainerCallback



In [43]:
# 1. Définir le modèle
model_name = "bigscience/bloomz-560m"

In [ ]:
wandb.login(key="")          
run = wandb.init(
    project="bloom-lora-quadro",
    name=f"run-{wandb.util.generate_id()}",
    config={
        "model": model_name,
        "r": 8,
        "lora_alpha": 16,
        "learning_rate": 3e-4,
        "epochs": 1,
        "max_length": 256
    }
)

# 3. Configuration GPU

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 GPU actif : {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")


🚀 GPU actif : CPU


# 4. Chargement du modèle et du tokenizer

In [46]:
model_name = "bigscience/bloomz-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 5. Chargement GPU standard (sans 4-bit pour Quadro M2000M)

In [47]:
foundation_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,  # float32 pour compatibilité
    device_map="auto"
)

# 6. Fonction de génération

In [48]:
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"].to(model.device),
        attention_mask=inputs["attention_mask"].to(model.device),
        max_new_tokens=max_new_tokens,
        repetition_penalty=1.5,
        early_stopping=True,
        eos_token_id=tokenizer.eos_token_id,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return outputs


# 7. Test modèle original

In [49]:
print("🧪 Test modèle original :")
input_sentences = tokenizer("I want you to act as a motivational coach. ", return_tensors="pt")
original_outputs = get_outputs(foundation_model, input_sentences, max_new_tokens=50)
print(tokenizer.batch_decode(original_outputs, skip_special_tokens=True))

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


🧪 Test modèle original :
['I want you to act as a motivational coach.  Be practical, not just about numbers.']


# 8. Chargement dataset ChatGPT prompts

In [50]:
dataset = load_dataset("fka/awesome-chatgpt-prompts")
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)


# 9. Préparation dataset (50 échantillons max pour 4GB VRAM)

In [51]:
train_sample = dataset["train"].select(range(50))
train_sample = train_sample.remove_columns('act')

#Création jeu de validation (20 prompts)
eval_sample = dataset["train"].select(range(50, 70))  # 20 prompts après les 50 d'entraînement
eval_sample = eval_sample.remove_columns('act')

# 10. Configuration LoRA optimisée 4GB VRAM

In [52]:
lora_config = LoraConfig(
    r=8,                    # Réduit pour 4GB VRAM
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="lora_only",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(foundation_model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 860,160 || all params: 560,001,024 || trainable%: 0.15359971913194215


# 11. Configuration entraînement

In [53]:
output_directory = "./quadro-lora-chatgpt"
os.makedirs(output_directory, exist_ok=True)

training_args = TrainingArguments(
    output_dir="./bloomz-lora-wandb",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    num_train_epochs=2,
    logging_steps=10,
    save_steps=50,
    eval_strategy="steps",
    eval_steps=25,              # évalue toutes les 25 steps
    load_best_model_at_end=True,
    report_to="wandb",        # ← envoie les logs à wandb
    run_name=run.name
)

class WandbCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, **kwargs):
        # Exemple : log perplexité finale
        if state.log_history:
            wandb.log({"final_perplexity": state.log_history[-1].get("eval_loss", 0)})

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_sample,
    eval_dataset=eval_sample,  # ← ajout du jeu de validation
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[WandbCallback()]   # ← ajout du callback
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


# 12. Entraînement

In [55]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_sample,
    eval_dataset=eval_sample,  # ← ajoute cette ligne
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

print("📊 Lancement de l'entraînement...")
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


📊 Lancement de l'entraînement...


c:\Users\Zbook\anaconda3\envs\gpu_env\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


TrainOutput(global_step=8, training_loss=3.3716042041778564, metrics={'train_runtime': 327.0138, 'train_samples_per_second': 0.306, 'train_steps_per_second': 0.024, 'total_flos': 20008230862848.0, 'train_loss': 3.3716042041778564, 'epoch': 2.0})

# 13. Sauvegarde

In [56]:
peft_model_path = os.path.join(output_directory, "lora_model")
trainer.model.save_pretrained(peft_model_path)
print("✅ Modèle sauvegardé !")

✅ Modèle sauvegardé !



# 14. Chargement et test modèle fine-tuné

In [57]:
loaded_model = PeftModel.from_pretrained(
    foundation_model,
    peft_model_path,
    is_trainable=False
)

# 14-bis. Test rapide avec prompts
print("\n🎯 Test rapide des prompts :")
test_prompts = [
    "Act as a motivational coach and give me a short tip:",
    "Write a creative slogan for coffee:",
    "Explain AI in one sentence:"
]

for p in test_prompts:
    inputs = tokenizer(p, return_tensors="pt").to(device)
    out = loaded_model.generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.8,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    print(f"\n📝 Prompt: {p}\n🔮 Generated: {tokenizer.decode(out[0], skip_special_tokens=True)}")

# 15. Fin wandb
wandb.finish()


print("\n🎯 Test modèle fine-tuné :")
input_sentences = tokenizer("I want you to act as a motivational coach. ", return_tensors="pt")
fine_tuned_outputs = get_outputs(loaded_model, input_sentences, max_new_tokens=50)
print(tokenizer.batch_decode(fine_tuned_outputs, skip_special_tokens=True))


🎯 Test rapide des prompts :

📝 Prompt: Act as a motivational coach and give me a short tip:
🔮 Generated: Act as a motivational coach and give me a short tip: Take your dog to the vet. If you don't see a problem in your dog, you might be able to get a dog that is sick, injured, or injured, and treat it for another reason

📝 Prompt: Write a creative slogan for coffee:
🔮 Generated: Write a creative slogan for coffee: "Just get a coffee"


wandb: ERROR The nbformat package was not found. It is required to save notebook history.



📝 Prompt: Explain AI in one sentence:
🔮 Generated: Explain AI in one sentence: AI is a search engine that analyzes a human body and uses machine learning to determine what is relevant to a user.


train/epoch,▁
train/global_step,▁
total_flos,20008230862848.0
train/epoch,2
train/global_step,8
train_loss,3.3716
train_runtime,327.0138
train_samples_per_second,0.306
train_steps_per_second,0.024


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🎯 Test modèle fine-tuné :
["I want you to act as a motivational coach.  Don't make excuses about what you're experiencing. Be available, but don't be inaccessible"]


In [58]:
trainer.model.save_pretrained("./final-lora-wandb")
tokenizer.save_pretrained("./final-lora-wandb")
wandb.finish()

Métrique	Valeur	Commentaire
train_loss	3.37	 Nette amélioration (vs 14.99 précédent) → le modèle apprend !
train_runtime	327 s (~5 min 30)	⏱️ + rapide que 488 s → optimisation mémoire réussie
train_steps_per_second	0.024	🏃 +50 % de vitesse (0.016 → 0.024)

test précédent (le learning-rate (3e-2).): 

Métrique	Valeur obtenue	Commentaire
train_loss	14.99	Très élevé → le modèle n’a pas encore appris grand-chose (valeur typique < 3-4 pour ce type de tâche).
train_runtime	488 s (~8 min)	Raisonnable pour 50 exemples sur Quadro M2000M.
train_samples_per_second	0.205	Faible, mais attendu sur GPU 4 GB sans optimisation agressive.
train_steps_per_second	0.016	Correspond à ~1 step toutes les 60 s → normal avec batch-size 4 et gradient-accumulation 4.

